In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Simulation of Conditioned Diffusions on Riemannian Manifolds

Mathias Højgaard Jensen and Stefan Sommer

# $S^2$

In [ ]:
%cd ..

In [ ]:
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

# geodesics
from src.Riemannian import geodesic
geodesic.initialize(M)

x = M.coordsf([0.,0.])

In [ ]:
# Brownian motion, coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M,do_chart_update=M.do_chart_update)

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# guide function
Cholesky = T.slinalg.Cholesky()
phi = lambda q,v: T.tensordot(Cholesky(M.gsharp(q)).T,M.StdLog(q,v).flatten(),(1,0))

# plot guiding field
M.newfig()
M.plot_field(lambda x: .2*M.StdLogf(x,tensor([0,0,-1])))
plt.show()

(Brownian_coords_guided,Brownian_coords_guidedf) = get_guided_likelihood(
    M,M.sde_Brownian_coords,phi,lambda x: Cholesky(M.gsharp(x)),
    use_charts=True,chart_update=M.chart_update_Brownian_coords)   

n_steps.set_value(1000)

w = M.Ff(M.Expf(x,tensor(np.array([.8,-.5]))))
(ts,xs,charts,log_likelihood,log_varphi) = Brownian_coords_guidedf(x,w,dWsf(M.dim.eval()))[:5]
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

# plot
newfig()
M.plot()
M.plot_path(zip(xs,charts))
M.plotx(x,color='r',s=150)
M.plotx(w,color='k',s=150)
plt.show()

# plot multiple bridges
N = 5
w = M.Ff(M.Expf(x,tensor(np.array([.1,-.8]))))
xss = tensor(np.zeros((N,n_steps.eval(),M.dim.eval())))
chartss = tensor(np.zeros((N,n_steps.eval(),x[1].shape[0])))
for i in range(N):
    (ts,xs,charts,log_likelihood,log_varphi) = Brownian_coords_guidedf(x,w,dWsf(M.dim.eval()))[:5]
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(x,color='r',s=100)
M.plotx(w,color='k',s=100)
plt.savefig('S2_bridges.pdf')
plt.show()

In [ ]:
# plot multiple bridges
N = 1
w = M.Ff(M.Expf(x,tensor(np.array([.1,-.8]))))
xss = tensor(np.zeros((N,n_steps.eval(),M.dim.eval())))
chartss = tensor(np.zeros((N,n_steps.eval(),x[1].shape[0])))
for i in range(N):
    (ts,xs,charts,log_likelihood,log_varphi) = Brownian_coords_guidedf(x,w,.0*dWsf(M.dim.eval()))[:5]
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(x,color='r',s=100)
M.plotx(w,color='k',s=100)
plt.savefig('S2_bridges.pdf')
plt.show()

In [ ]:
## Sample Data

var = .4
n_steps.set_value(100)

N_samples = 256

samples = tensor(np.zeros((N_samples,M.dim.eval())))
charts = tensor(np.zeros((N_samples,x[1].shape[0])))
for i in range(N_samples):
    (ts,xs,_charts) = M.Brownian_coordsf(x,var*dWsf(M.dim.eval()))
    samples[i] = xs[-1]
    charts[i] = _charts[-1]

obss = np.hstack((samples,charts))
    
# plot
newfig()
M.plot()
for i in range(N_samples):
    M.plotx((samples[i],charts[i]))
plt.show()

In [ ]:
# setup for ML mean estimation

options = {}
options['samples_per_obs'] = 1
options['epochs'] = 75
options['learning_rate'] = .5e-0
options['varphi_update_rate'] = 1.
options['initial'] = [M.update_coordsf(x,M.chartf())[0]+.1*np.random.randn(M.dim.eval())]
options['verbose'] = True

# transition density etc.
q0 = M.sym_coords()
v = M.sym_coords()
chart = M.sym_chart()
thetas = (q0,)
_log_p_Tf = theano.function([q0,chart,v],log_p_T((q0,chart),v,dWs(M.dim),Brownian_coords_guided,phi,options,sde=M.sde_Brownian_coords,F=M.F,use_charts=True))
_dlog_p_Tf = theano.function([q0,chart,v],dlog_p_T(thetas,(q0,chart),v,dWs(M.dim),Brownian_coords_guided,phi,options,sde=M.sde_Brownian_coords,F=M.F,use_charts=True))
_p_Tf = theano.function([q0,chart,v],T.exp(log_p_T((q0,chart),v,dWs(M.dim),Brownian_coords_guided,phi,options,sde=M.sde_Brownian_coords,F=M.F,use_charts=True)))
log_p_Tf = lambda x,v: _log_p_Tf(x[0],x[1],v)
dlog_p_Tf = lambda x,v: _dlog_p_Tf(x[0],x[1],v)
p_Tf = lambda x,v: _p_Tf(x[0],x[1],v)

v = x
print(x)
print(v)
%time print(log_p_Tf(x,v[0]))
%time print(p_Tf(x,v[0]))
%time print(dlog_p_Tf(x,v[0]+.1*np.random.randn(M.dim.eval())))

from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (obs,seed) = pars
    if seed:
        srng.seed(seed)
    v = (obs[0:M.dim.eval()],obs[M.dim.eval():])
    q = M.coordsf(thetas[0])
    qchartv = M.update_coordsf(q,v[1])
    res = dlog_p_Tf(qchartv,v[0])
    return (res[0],M.update_vectorf(qchartv,q,res[1]))

def update_thetas(thetas, dthetas):
    q = (thetas[0],M.chartf())
    
    new_q = q[0]+options['learning_rate']*np.dot(M.gsharpf(q),dthetas[0]) # use Riemannian g-gradient
    
    return (new_q,)

In [ ]:
pts = 20
phi, theta = np.meshgrid(0.,np.linspace(np.pi/2,-np.pi/2+1e-2,pts))
phitheta = np.vstack([phi.ravel(), theta.ravel()]).T
xs = np.apply_along_axis(M.F_sphericalf,1,phitheta)

# plot points
newfig()
M.plot()
M.plotx(x,color='r',s=100)
for i in range(xs.shape[0]):
    M.plotx((M.invFf((xs[i],x[1])),x[1]))

# compute transition density
fs = np.apply_along_axis(lambda v: p_Tf(x,M.invFf((v,x[1]))),1,xs)

newfig2d()
plt.plot(-theta,fs)

In [ ]:
# transition density plot
newfig()
plot_sphere_f(M,lambda v: p_Tf(x,M.invFf((v,x[1]))),alpha=.8,pts=100,parallel=True,vmin=0.)
# plt.savefig('sphere_pT.pdf') # transition density plot
plt.show()

In [ ]:
# run MLE
n_steps.set_value(50)
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss,llog_p_T,update_thetas,options)

# plot
plt.plot(range(options['epochs']),log_likelihoods)
# plt.savefig('ML_likelihoods.pdf')
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
# plt.savefig('ML_thetas.pdf')
plt.show()

M.newfig()
M.plot()
M.plotx(M.coordsf(options['initial'][0]),color='k',s=100)
M.plotx(M.coordsf(thetass[0][-1]),color='b',s=100)
M.plotx(x,color='r',s=100)
M.plot_path([M.coordsf(thetass[0][i]) for i in range(options['epochs'])],color='b',linewidth=2.5)

plt.savefig('MLmean_iterations.pdf')
plt.show()

n_steps.set_value(100)